In [1]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.3 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install rank_bm25

In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))


CUDA available: True
CUDA device count: 2
Current device: 0
Device name: Tesla T4


In [4]:
import sqlite3
import numpy as np
import faiss
import torch
from datasets import load_dataset
from transformers import DPRQuestionEncoder, DPRContextEncoder, AutoTokenizer, pipeline
from rank_bm25 import BM25Okapi

In [5]:
# 1️⃣ Load pre-trained models
device = "cuda"
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
qa_pipeline = pipeline("question-answering", model="KienLe21/demo_qa_model", device=0)

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
# 2️⃣ Load and process dataset
squad = load_dataset("squad_v2", split="train").select(range(10000))
celebrity_data = [{"context": d["context"]} for d in squad]

docs = [c["context"] for c in celebrity_data]


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [9]:
import sqlite3

# Kết nối SQLite
db_conn = sqlite3.connect("embeddings.db")
cursor = db_conn.cursor()

# # Xóa dữ liệu cũ để tránh lỗi ID trùng
# cursor.execute("DELETE FROM contexts")
# db_conn.commit()

# Tạo bảng nếu chưa có
cursor.execute("CREATE TABLE IF NOT EXISTS contexts (id INTEGER PRIMARY KEY, text TEXT)")

# Chèn dữ liệu mới
for i, doc in enumerate(docs):
    cursor.execute("INSERT INTO contexts (id, text) VALUES (?, ?)", (i, doc))

db_conn.commit()
db_conn.close()


In [10]:
# Kết nối với database SQLite
db_conn = sqlite3.connect("embeddings.db")
cursor = db_conn.cursor()

# Kiểm tra số lượng dòng trong database
cursor.execute("SELECT COUNT(*) FROM contexts")
row_count = cursor.fetchone()[0]
print(f"Total rows in database: {row_count}")

# Hiển thị 5 dòng đầu tiên để kiểm tra
cursor.execute("SELECT * FROM contexts LIMIT 5")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Đóng kết nối
db_conn.close()

Total rows in database: 10000
(0, 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".')
(1, 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose

In [11]:
# 4️⃣ BM25 Indexing
bm25 = BM25Okapi([tokenizer.tokenize(doc) for doc in docs])

In [13]:
embedding_size = 768  # DPR dimension

# 🔹 Tạo GPU resource và chuyển index sang GPU
res = faiss.StandardGpuResources()  
faiss_index_cpu = faiss.IndexFlatL2(embedding_size)  
faiss_index = faiss.index_cpu_to_gpu(res, 0, faiss_index_cpu)  # 0 là GPU ID

embeddings = []
for doc in docs:
    inputs = tokenizer(doc, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)
    with torch.no_grad():
        emb = context_encoder(**inputs).pooler_output.cpu().numpy()
    
    embeddings.append(emb)
    faiss_index.add(emb)  # 🔹 Thêm embedding vào index GPU


In [14]:
def query_celebrity(question):
    inputs = tokenizer(question, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        question_emb = question_encoder(**inputs).pooler_output.cpu().numpy()
    
    # 🔹 Tăng số lượng context lấy từ BM25
    top_n_bm25 = 25  # Thử lấy nhiều context hơn từ BM25
    bm25_scores = bm25.get_scores(tokenizer.tokenize(question))
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_n_bm25]  
    top_bm25_contexts = [docs[i] for i in top_bm25_indices]

    # 🔹 Dùng FAISS để tìm context tốt nhất trong tập BM25
    top_embeddings = []
    for doc in top_bm25_contexts:
        inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            emb = context_encoder(**inputs).pooler_output.cpu().numpy()
        top_embeddings.append(emb)
    
    faiss_subset_index = faiss.IndexFlatL2(embedding_size)
    faiss_subset_index.add(np.vstack(top_embeddings))
    
    _, faiss_results = faiss_subset_index.search(question_emb, k=1)
    
    best_context = top_bm25_contexts[faiss_results[0][0]]  
    answer = qa_pipeline(question=question, context=best_context)
    
    return {"answer": answer["answer"], "score": answer["score"], "context": best_context}


In [16]:
# 7️⃣ Example query
query = "When did Beyoncé release her fifth studio album?"
response = query_celebrity(query)
print(response)


{'answer': 'December 13, 2013', 'score': 0.9463059902191162, 'context': 'On December 13, 2013, Beyoncé unexpectedly released her eponymous fifth studio album on the iTunes Store without any prior announcement or promotion. The album debuted atop the Billboard 200 chart, giving Beyoncé her fifth consecutive number-one album in the US. This made her the first woman in the chart\'s history to have her first five studio albums debut at number one. Beyoncé received critical acclaim and commercial success, selling one million digital copies worldwide in six days; The New York Times noted the album\'s unconventional, unexpected release as significant. Musically an electro-R&B album, it concerns darker themes previously unexplored in her work, such as "bulimia, postnatal depression [and] the fears and insecurities of marriage and motherhood". The single "Drunk in Love", featuring Jay Z, peaked at number two on the Billboard Hot 100 chart. In April 2014, after much speculation in the weeks befo